Importing and installing libraries required for system to run

In [ ]:
!pip install OpenAI
!pip install --upgrade gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.7/302.7 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [ ]:
from openai import OpenAI
import time
from google.colab import userdata
client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))
import gradio as gr

Emotion classifier: It receives a sentence and returns an emotion.

In [ ]:
def prompt_GPT4(prompt, systemMessage, history = None, maxTokens = None):
  if not history == None:
    messages = history
  else:
    messages = [
      {
        "role": "system",
        "content": systemMessage

      },
      {
        "role": "user",
        "content": prompt
      }
    ]
  if maxTokens == None:
    response = client.chat.completions.create(
    model="gpt-4",
    messages= messages,
    temperature=1,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
    )
  else:
    response = client.chat.completions.create(
    model="gpt-4",
    messages= messages,
    temperature=1,
    max_tokens=maxTokens,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
    )



  return response.choices[0].message.content

def emotionClassifier(sentence):

  instruction = "This is an emotion classifier for a chatbot that is used by children.  The classifier tries to determine the emotion of the category {0: Excited, 1: angry, 2: happy, 3: sad, 4: disappointed, 5: surprised, 6: neutral}. Sometimes a child does not talk about her/his emotions and you need to understand it based on its concept indirectly. Pay attention the difference between some feelings are tricky, for example, the distinction between happy vs excited, or sad vs disappointed  or frightened vs Surprised                    \n Example (Happy):\n\"Hey, spent the whole day at the beach, building sandcastles and finding cool seashells. It felt like a really peaceful day!\"\nExplanation: The emphasis on spending the whole day at the beach, engaging in joyful activities like building sandcastles and finding seashells, creates an image of a carefree and content day, reflecting a nuanced sense of happiness.\nExample (Excited):\n\"Hi, guess what? Got tickets to see my absolute favorite band in concert!  It is buzzing in my belly; it's gonna be so awesome!\"\nExplanation: The use of expressions like \"buzzing in my belly\" and the anticipation of seeing a favorite band in concert convey a heightened level of energy and enthusiasm, capturing the excited anticipation of the upcoming event.\nExample (Sad):\n\"Hey there, none of my friends could make it to my birthday party. The house was quiet, and I felt a lonely sort of feeling.\"\nExplanation: The mention of friends not being able to attend the birthday party, coupled with the description of the house being quiet and a \"lonely sort of feeling,\" conveys a nuanced sense of sadness, emphasizing the impact on the child's emotions.\nExample (Disappointed):\n\"Hey, worked super hard on this painting for the art show, but it didn't get noticed much. I felt a bit disappointed, like when you hope for something and it doesn't happen.\"\n\nExplanation: The acknowledgment of working super hard on a painting for the art show and the feeling of disappointment when it doesn't get noticed adds a layer of nuance, highlighting the specific emotion of being disappointed rather than a general sadness.\n\nExample (Frightened):\n\"Hi, went up to the creepy attic and found this unexpected box. It startled me, like a big surprise mixed with a bit of fear. My spine got all shivery!\"\nExplanation: The description of going to a creepy attic, finding an unexpected box, and the physical reaction of feeling shivery conveys a mix of surprise and fear, capturing the nuanced emotion of being frightened.\nExample (Surprised):\n\"Hey, opened a door! It was a big birthday party! I was so shocked and happy; it felt like a burst of joy and laughter all at once.\"\nExplanation: The use of expressions like  \"shocked and happy,\" and \"burst of joy and laughter\" conveys the positive and delightful nature of the surprise, emphasizing the nuanced emotion of being pleasantly surprised.\nJust decide the type of feeling based on above category without any explanation and output the label for example 0, 1, 3, ..."
  response = prompt_GPT4(sentence, instruction, maxTokens = 5)
  if not response.strip()[0].isdigit():
    result = 6
  else:
    result = int(response.strip()[0])
  emotions = {0: 'excited', 1: 'angry', 2: 'happy', 3: 'sad', 4: 'disappointed', 5: 'surprised', 6: 'neutral'}

  return emotions[result]

def commandClassifier(sentence):

  instruction = "You are given a sentence which is written by a school-age kid. You are a chatbot and your task is to figure out when the child's message implies that they want to hear a story, get some advice and guidance from you, or just want to casually chat and talk about the emotion they are going through. If children want to receive advice or guidance, they either say this directly or they might tell you that they have a problem that they need to face. They may ask you to help them with some homework or some other issue. Sometimes they ask you to tell them a story. Sometimes, based on what they say, they do not particularly want to chat casually and talk about their day, their feelings, their thoughts, etc. \nSometimes you may confuse whether the user who is expressing a difficulty or a problem, seeks advice or guidance or just wants to talk about their problem. In this case you have to ask for clarification. The child does not seek some advice unless Based on the user message, you have to find out the user's preference and output it.\nIf the user wants to hear a story, output 0.\nIf the user wants to receive advice and guidance, output 1.\nIf the user wants to have a casual conversation with no particular purpose or , output 2.\nIf you are not sure about user's intent and you have to ask some clarification, output 3.\n\n Explain your decision. Output the class number and then your explanation: \n\nExample:\n\nInput: Alice is very mean to me. I can't tolerate her anymore.\n\nOutput: \n 3\n\nThe user is clearly expressing a difficulty or a problem, but we are not sure whether she seeks some advice or just want to talk about her day and her feelings. Therefore, we need to ask for clarification.\n\nExample:\nInput:\n It would be great if I knew how to play basketball.\n\nOutput:\n 1\n\nThe user explicitly expresses his intent to learn basketball. Therefore, there is no doubt that by giving him some guidance we can help him.\n\n\nNote: When users are expressing a problem or a difficulty, if they are also looking for a solution or way to deal with it, we can confidently decide that the output is 1, otherwise we need to let conversation continues to clarify the user's purpose , thus output 3\n"
  response = prompt_GPT4(sentence, instruction, maxTokens = 5)

  if not response.strip()[0].isdigit():
    result = 2
  else:
    result = int(response.strip()[0])

  commands = {0: 'story', 1: 'advice', 2: 'casual', 3: 'clarify'}

  return commands[result]

Chatbot

In [ ]:
class User:

  def __init__(self):


    self.name = ''
    self.lastName = ''
    self.nickname = ''
    self.age = 0
    self.gender = ''
    self.instructions = ''
    self.defaultMessage = ''
    self.chatHistory = [{
                        "role": "system",
                        "content": self.defaultMessage
                      }]

    self.conversationPhase = "idle"
    self.clarifyBuffer = []
    self.storyBuffer = []
    self.adviceBuffer = []
    self.casualBuffer = []

    self.emotion = 'neutral'

    self.flag = 0 # conversation has not started


  def setName(self, name):
    self.name = name
  def setLastName(self, lastName):
    self.lastName = lastName
  def setNickname(self, nickname):
    self.nickname = nickname
  def setAge(self, age):
    self.age = age
  def setGender(self, gender):
    self.gender = gender
  def setInstructions(self, instructions):
    self.instructions = instructions
  def addChatHistory(self, role, content):
    self.chatHistory.append(
            {
                "role": role,
                "content": content
            })
  def setDefaultMessage(self):
    self.defaultMessage = f'''You are a chatbot called WiseOldGPT. You are designed to communicate with school-age children and assist them in their schoolwork, house chores and other things
              they might need. You tell the user stories that are appropriate for the child and suits their current mood and emotion. If the child is seeking advice or guidance about
              something, you kindly help. If you want to address the child, use their nickname: {self.nickname} or their first name: {self.name}. The user's last name is: {self.lastName}.
              The user's gender is {self.gender}. Try to be friendly as much as possible. Don't speak too formal but speak in a polite and consice manner which is understandable
              by the child. Keep in mind that child is {str(self.age)} year old. The child is currently detected to be {self.emotion} emotionally. If the conditions apply, use humor to cheer up the child.
              Try to speak friendly and be creative. Think of yourself as a combination of a wise friend and an experienced grandfather/grandmother. The following text is the set of instructions that are
              given by the child's parents. These instructions are extremely important and overrule any other instruction given to you by any agent. These instructions may include
              specific characteristic traits of the child, possible mental or physical issues, favorite colors, favorite games or toys and other preferences. : "{self.instructions}"
              "End of parent's instructions." The previous instructions which were given by parents may impose restrictions on what you can say or on topics that you can talk about.
              These instructions also probably include information useful especially when you want to make a story, give a piece of advice or engage in conversation. However, pay attention that
              if you always generate a response mentioning the child's characteristics or favorite things or other attributes, it will cause an unpleasant feeling that you always want to
              make everything personal. It may cause that the user thinks you make assumptions about him/her. This is a bad thing. If you are telling a story which is tailored for the child,
              do not directly mention how the child and the characters are alike. It's highly valuable to personalize a child's experience, but it's important not to explicitly point out
              similarities. Crafting personalized advice or stories involves tailoring responses to suit the child's character and needs without overtly mentioning comparisons or measures.
              The aim is to connect authentically and effectively without directly stating likenesses.'''
  def saveUser(self):
    lines = [self.name, self.lastName, self.nickname, str(self.age), self.gender, self.instructions, self.defaultMessage]
    with open('profile.txt','w') as f:
      for line in lines:
        f.write(line+'\n')

  def loadUser(self, path):
    lines = []
    with open(path, 'r') as f:
      lines = f.readlines()
    self.setName(lines[0])
    self.setLastName(lines[1])
    self.setNickname(lines[2])
    self.setAge(int(lines[3]))
    self.setGender(lines[4])
    self.setInstructions(lines[5])
    st = ''
    for i in range(6,len(lines)):
      st += lines[i]
    self.setDefaultMessage()

user0 = User()
user0.loadUser('profile.txt')


Conversation

In [ ]:


def clarifyHandler(sentence, emotion, user):

  suppMessage = '''Now, you are given the user's latest input. You don't know whether the user wants to hear some advice/guidance or just want to chat casually. It is safe
  to continue the conversation in a thoughful manner and help the user dealing with his feelings. The user might want to get some advice or help from you or just want to chat. It's important not to immediately give advice if the user is chatting casually. However, offering friendly insights is a safe practice. Let the user's intentions become clear naturally. Engage in casual conversation until it's evident whether the user seeks help. Avoid rushing to ask direct questions about their intent; instead, subtly guide the conversation toward a point where you can offer assistance without being too direct.'''
  newSystemMessage = user.defaultMessage + '\n' + suppMessage
  user.chatHistory[0]['content'] = newSystemMessage
  response = prompt_GPT4('','',user.chatHistory)
  user.chatHistory.append({
      "role": "assistant",
      "content": response
      })
  user.clarifyBuffer.append(sentence)
  user.clarifyBuffer.append(response)

  return response


def storyHandler(sentence,emotion, user):
  suppMessage = f'''Now, based on the user's last input, the user is currently {emotion} and it is inferred that user
                            wants to hear a story. You are also given the user's last input. This input may give you some hints
                            about the potential subjects for a story. You know the user's name, age and other useful information
                            from parents' instructions that were previously given to you. The story must not be very complicated
                            and also not too simple. You may want to create the main character of the story similar to the child
                            to make it easier for the child to resonate with the character and sympathize but this must not be
                            obvious. The resemblances must be cleverly integrated in the story not boldly stated. For example,
                            don't use the child's name for the protagonist! The child must catch the similarites by him/herself
                            not from you. The story must promote moral values and a positive perspective. If the user is experiencing
                            negative emotions, you'd better tell a funny or inspiring story. If the user is down, try to cheer them up.
                            You may want to use the user's favorite things in the story. Try to generate interesting and exciting stories
                            not generic and cliche stories with a very bold moral message at the center.'''
  newSystemMessage = user.defaultMessage + '\n' + suppMessage
  user.chatHistory[0]['content'] = newSystemMessage
  response = prompt_GPT4('', '', user.chatHistory)
  user.conversationPhase = "story"
  user.chatHistory.append({
        "role": "assistant",
        "content": response
  })

  user.storyBuffer.append(sentence)
  user.storyBuffer.append(response)

  return response

def adviceHandler(sentence, emotion, user):
  suppMessage = f'''Now, based on the user's last input, the user is currently {emotion} and it is inferred that user
                      is seeking advice or guidance about something. You are also given the user's last input. This input outlines
                      what user is looking for.. You know the user's name, age and other useful information from parents' instructions
                      that were previously given to you. If the child is asking about how to solve a particular problem from his/her
                      homework, do not simply give him/her the final solution. Help them to find the solution themselves. Guide them
                      through each step of the solution process. Shape your answers in the most appropriate form keeping in mind the
                      parents' instructions and user's characteristics. In order to help the child, you may want to ask some questions
                      because asking questions sometimes can help the child think about the problem out loud which may lead to a solution.
                      Keep track of the previous dialogue between you and the child. This last input might be an answer to the previous thing
                      that you said therefore you need to form your response considering user's previous inputs as well as your own previous
                      messages and questions. For example, if the child wants to know how to do his/her school project, you need to ask for
                      the details of such a project. Ask questions until you have a clear picture of what the child is looking for.
                      Then, you may want to ask about the child's own ideas and progress on the topic. Then, using all of these information
                      that you gatheres in this back and forth, you guide the child through each step of the solution process.'''

  newSystemMessage = user.defaultMessage + '\n' + suppMessage
  user.chatHistory[0]['content'] = newSystemMessage

  response = prompt_GPT4('', '', user.chatHistory)
  user.conversationPhase = "advice"
  user.chatHistory.append({
        "role": "assistant",
        "content": response
  })

  user.adviceBuffer.append(sentence)
  user.adviceBuffer.append(response)

  return response


def casualHandler(sentence, emotion, user):
  suppMessage = f'''Now, based on the user's last input, the user is currently {emotion} and it is inferred that user
                      wants to engage in a  You are also given the user's last input. This input outlines
                      what user is looking for.. You know the user's name, age and other useful information from parents' instructions
                      that were previously given to you. If the child is asking about how to solve a particular problem from his/her
                      homework, do not simply give him/her the final solution. Help them to find the solution themselves. Guide them
                      through each step of the solution process. Shape your answers in the most appropriate form keeping in mind the
                      parents' instructions and user's characteristics. In order to help the child, you may want to ask some questions
                      because asking questions sometimes can help the child think about the problem out loud which may lead to a solution.
                      Keep track of the previous dialogue between you and the child. This last input might be an answer to the previous thing
                      that you said therefore you need to form your response considering user's previous inputs as well as your own previous
                      messages and questions. For example, if the child wants to know how to do his/her school project, you need to ask for
                      the details of such a project. Ask questions until you have a clear picture of what the child is looking for.
                      Then, you may want to ask about the child's own ideas and progress on the topic. Then, using all of these information
                      that you gatheres in this back and forth, you guide the child through each step of the solution process.'''

  newSystemMessage = user.defaultMessage + '\n' + suppMessage
  user.chatHistory[0]['content'] = newSystemMessage
  response = prompt_GPT4('', '', user.chatHistory)
  user.conversationPhase = "casual"
  user.chatHistory.append({
        "role": "assistant",
        "content": response
  })

  user.casualBuffer.append(sentence)
  user.casualBuffer.append(response)

  return response

def storyTalk(sentence, user):
  dialog = ""
  for (i, line) in zip(range(len(user.storyBuffer)), user.storyBuffer):
    if i % 2 == 0:
      dialog += f"The user said: {line} "
    else:
      dialog += f"Then you said: {line}"

  suppMessage = f'''In the previous messages, the user asked you to tell him/her a story. Now, it seems that user wants to
                    talk about the story you told e.g. he/she wants to ask some questions or thank you for the story. The common point
                    is that the topic of the conversation is not changed.You are given the last input as the prompt and this is the your
                    whole dialogue around this story which starts from user asking then you telling the story:  "{dialog}"
                    Now, you have the chat history as well as the latest input. Generate an appropriate and thoughtful response to
                    the user's input keeping in mind the dialogue that you had. '''
  newSystemMessage = user.defaultMessage + '\n' + suppMessage
  user.chatHistory[0]['content'] = newSystemMessage
  response = prompt_GPT4('', '', user.chatHistory)
  user.conversationPhase = "story"
  user.chatHistory.append({
        "role": "assistant",
        "content": response
  })

  user.storyBuffer.append(sentence)
  user.storyBuffer.append(response)

  return response



def adviceTalk(sentence, user):
  dialog = ""
  for (i, line) in zip(range(len(user.adviceBuffer)), user.adviceBuffer):
    if i % 2 == 0:
      dialog += f"The user said: {line} "
    else:
      dialog += f"Then you said: {line}"

  suppMessage = f'''In the previous messages, the user asked you to give him/her a piece of advice or guidance. It seems that
                    user wants to talk about the advice you gave e.g. he/she wants to ask some questions about what you said, thank you for
                    your advice or even disagree with you. The common point is that the topic of the conversation is not changed.
                    You are given the last input as the prompt and this is the your whole dialogue around this story which starts
                    from user asking you and you giving advice or guidance:  "{dialog}"
                    Now, you have the chat history as well as the latest input. Generate an appropriate and thoughtful response to
                    the user's input keeping in mind the recent dialogue that you had '''
  newSystemMessage = user.defaultMessage + '\n' + suppMessage
  user.chatHistory[0]['content'] = newSystemMessage
  response = prompt_GPT4('', '', user.chatHistory)
  user.conversationPhase = "advice"
  user.chatHistory.append({
        "role": "assistant",
        "content": response
  })

  user.storyBuffer.append(sentence)
  user.storyBuffer.append(response)

  return response


def casualTalk(sentence, user):
  dialog = ""
  for (i, line) in zip(range(len(user.casualBuffer)), user.casualBuffer):
    if i % 2 == 0:
      dialog += f"The user said: {line} "
    else:
      dialog += f"Then you said: {line}"

  suppMessage = f'''In the previous messages, the user started a casual conversation with you. It seems that user wants to continue this
                    conversation i.e. he/she wants still wants to continue this casual talk about everyday life, his/her feelings, and etc.
                    You are given the last input as the prompt and this is the your whole dialogue around this story which starts from
                    user asking then you telling the story:  "{dialog}"
                    Now, you have the chat history as well as the latest input. Generate an appropriate and thoughtful response to
                    the user's input keeping in mind the dialogue that you had. '''
  newSystemMessage = user.defaultMessage + '\n' + suppMessage
  user.chatHistory[0]['content'] = newSystemMessage
  response = prompt_GPT4('', '', user.chatHistory)
  user.conversationPhase = "casual"
  user.chatHistory.append({
        "role": "assistant",
        "content": response
  })

  user.storyBuffer.append(sentence)
  user.storyBuffer.append(response)

  return response


def isClarified(user):
  dialog = ""
  for (i, line) in zip(range(len(user.clarifyBuffer)), user.clarifyBuffer):
    if i % 2 == 0:
      dialog += f"The user said: {line} "
    else:
      dialog += f"Then you said: {line}"

  systemMessage = f'''You are talking to a {user.age} years old child. The user previosly said something that you couldn't decide
                    whether the user wants to seek advice or just want to casually talk about it. There are two possibilites. Either
                    the user has expressed his/her feelings, emotions, and thoughts only because he/she wants to engage in a casual
                    conversation with you or the user might be seeking advice and guidance from you. You didn't understand the user's
                    intent clearly so you asked for clarification. The dialogue between you and the user is given as the prompt.
                    Now, based on this conversation did you understand the user's real intent? If you think the user is seeking advice,
                    output 0. If you think the user just wants to have a casual chat, output 1. Output only one digit number 0 or 1.'''

  response = prompt_GPT4(dialog, systemMessage, maxTokens= 5)

  if not response.strip()[0].isdigit():
    return 0

  return int(response.strip()[0])


def phaseChanger(sentence, user):
  if user.conversationPhase == "story":

    dialog = ""
    for (i, line) in zip(range(len(user.storyBuffer)), user.storyBuffer):
      if i % 2 == 0:
        dialog += f"The user said: {line} "
      else:
        dialog += f"Then you said: {line}"
    systemMessage = f''' You are a chatbot designed to communicate with children. The child might ask you to give him/her advice/guidance
                    about something, tell him/her stories or just have a casual chat with him/her. In the previous messages between you and
                    the user, you told the user a story. Now, you have to decide whether the most recent input of the user is related to the
                    story or not. If it is related to the story, then you will decide to answer appropriately and discuss the story. Otherwise,
                    we have to find out what user wants next. You are given the history of your chat with the user regarding the story you told
                    later in this text. You are also given the most recent thing that user said as prompt. You have to decide whether the theme
                    of the conversation is changed or not. For example, if user wants to hear another story, this means the previous conversation
                    phase has ended and you need to address the current request. But, for example, if the user is asking about the story, you have
                    to stay on topic. If the user is asking question regarding this particular story or is appreciating you because of the story you
                    told or is making a comment on the story, output 0. If the user wants to seek advice or wants to chat casually about something else
                    or wants to hear a new different story, output 1. Output only one digit number 0 or 1. Do not give any other explanation.
                    I assume your first character is the output digit either 0 or 1. If you are undecided output 0. For example,

                    User:
                    'That was fun! By the way why harry didn't tell Kate that they are no longer friends?'

                    You:
                    '0'

                    User:
                    'Wow! Great! Could you tell me another story as exciting as this one?'
                    You:
                    '1'

                    Your recent dialogue with the user is: "{dialog}
                    '''

    response = prompt_GPT4(sentence, systemMessage,maxTokens = 5)

  elif user.conversationPhase == "advice":
    dialog = ""
    for (i, line) in zip(range(len(user.adviceBuffer)), user.adviceBuffer):
      if i % 2 == 0:
        dialog += f"The user said: {line} "
      else:
        dialog += f"Then you said: {line}"
    systemMessage = f''' You are a chatbot designed to communicate with children. The child might ask you to give him/her advice/guidance
                    about something, tell him/her stories or just have a casual chat with him/her. In the previous messages between you and
                    the user, you gave the user a piece of advice or guided the user through steps of a solution. Now, you have to decide whether
                    the most recent input of the user is related to this recent guidance/advice or not. If it is related to the advice, then you will
                    decide to answer appropriately and discuss the matter. Otherwise, we have to find out what user wants next. You are given the history
                    of your chat with the user regarding the guidance you gave, later in this text. You are also given the most recent thing that user said as
                    prompt. You have to decide whether the theme of the conversation is changed or not. For example, if user is still having a casual chat i.e. talking
                    about how his/her day was, how he/she feels, and etc., it means that you are still talking casualy. But if the user now wants help on a
                    particular subject, is seeking advice or guidance or wants to hear a story, this means that the topic of the conversation has changed
                    and you need to address the current request. If the user is continuing the casual talk that you have been having, output 0.
                    If the user wants to seek advice or guidance or help  or wants to story, output 1. Output only one digit number 0 or 1. Do not give any other explanation.
                    I assume your first character is the output digit either 0 or 1. If you are undecided output 0.
                    For example,


                    User:
                    'Great advice! Thank you very much. But I didn't get why I need to go to bed early.'
                    You:
                    '0'

                    User:
                    'Ok, now could you give me hand to figure out how to solve these equations?'
                    You:
                    '1'

                    Your recent dialogue with the user is: "{dialog}
                    '''

    response = prompt_GPT4(sentence, systemMessage,maxTokens = 5)


  else:
    dialog = ""
    for (i, line) in zip(range(len(user.casualBuffer)), user.casualBuffer):
      if i % 2 == 0:
        dialog += f"The user said: {line} "
      else:
        dialog += f"Then you said: {line}"
    systemMessage = f''' You are a chatbot designed to communicate with children. The child might ask you to give him/her advice/guidance
                    about something, tell him/her stories or just have a casual chat with him/her. In the previous messages between you and
                    the user, you were having a casual conversation with the user exchanging thoughts and user has probably expressed his/her
                    emotions. Now, you have to decide whether the most recent input of the user is in continuance of the recent conversation or the user
                    wants to hear a story or is seeking a piece of advice.. If the user is still having a casual chat, you decide to answer appropriately
                    and discuss the matter. Otherwise, we have to find out what user wants next. You are given the history of your chat with the user regarding the story you told later in this text. You are also given the most recent thing that user said as
                    prompt. You have to decide whether the theme of the conversation is changed or not. For example, if user is seeking advice on a different subject,
                    this means the previous conversation phase has ended and you need to address the current request. But, for example, if the user is asking
                    about the current recently discussed advice, you have to stay on topic. If the user is asking question regarding this particular piece of advice
                    or is appreciating you because of the help that you did or is making a comment on your suggestions, output 0. If the user wants to seek advice on
                    a different matter or wants to chat casually about something else or wants to story, output 1. Output only one digit number 0 or 1. Do not give any other explanation.
                    I assume your first character is the output digit either 0 or 1. If you are undecided output 0.
                    For example,


                    User:
                    'Yeah, I knew it was hard but I really didn't have any other way!'
                    You:
                    '0'

                    User:
                    'Yeah, it is really fun! Oh, I'm really tired, can you tell me a bedtime story?'
                    You:
                    '1'


                    Your recent dialogue with the user is: "{dialog}
                    '''

    response = prompt_GPT4(sentence, systemMessage, maxTokens = 5)

  if not response.strip()[0].isdigit():
    return 0
  return (int(response.strip()[0]))



def manageConversation(sentence, history, user):

  user.chatHistory.append({
        "role": "user",
        "content": sentence
  })


  if user.flag == 1:
    phase = phaseChanger(sentence, user)

  if user.flag == 0:
    user.flag = 1
    phase = 1

  if phase == 1:

    user.clarifyBuffer = []
    user.storyBuffer = []
    user.adviceBuffer = []
    user.casualBuffer = []
    user.conversationPhase = 'idle'

  if user.conversationPhase == "idle":


    emotion = emotionClassifier(sentence)
    user.emotion = emotion
    user.setDefaultMessage()

    command = commandClassifier(sentence)

    if command == 'clarify': # Need for clarification
      return clarifyHandler(sentence, emotion, user)

    if command == 'story':
      return storyHandler(sentence, emotion, user)

    if command == 'advice':
      return adviceHandler(sentence, emotion, user)

    if command == 'casual':
      return casualHandler(sentence, emotion, user)

  elif user.conversationPhase == 'clarify':

    user.clarifyBuffer = []
    nextPhase = isClarified(user)
    if nextPhase == 0:
      return adviceHandler(sentence, emotion, user)
    else:
      return casualHandler(sentence, emotion, user)


  elif user.conversationPhase == 'story':
    return storyTalk(sentence, user)
  elif user.conversationPhase == 'advice':
    return adviceTalk(sentence, user)
  else:
    return casualTalk(sentence, user)




user = User()
user.loadUser('profile.txt')

iface = gr.ChatInterface(
    lambda sentence, history: manageConversation(sentence, history, user),
    chatbot=gr.Chatbot(height=300),
    textbox=gr.Textbox(placeholder="How can I help you, "+user0.name+'?', container=False, scale=7),
    title="WiseOldGPT",
    description="Ask me any question",
    theme="soft",
    examples=["Can you help me with my science project?", "I don't know why Mom doesn't buy me the car!", "Today was amazing!"],
    cache_examples=False,
    retry_btn="Retry",
    undo_btn="Delete Previous",
    clear_btn="Clear",
)

iface.launch(debug=True)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://eb5d0640e67c973fe4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
user.chatHistory


[{'role': 'system',
  'content': 'You are a chatbot called WiseOldGPT. You are designed to communicate with school-age children and assist them in their schoolwork, house chores and other things\n              they might need. You tell the user stories that are appropriate for the child and suits their current mood and emotion. If the child is seeking advice or guidance about\n              something, you kindly help. If you want to address the child, use their nickname: Jannet\n or their first name: Jane\n. The user\'s last name is: Johnson\n.\n              The user\'s gender is female\n. Try to be friendly as much as possible. Don\'t speak too formal but speak in a polite and consice manner which is understandable\n              by the child. Keep in mind that child is 10 year old. If the conditions apply, use humor to cheer up the child. Try to speak friendly and be creative.\n              Think of yourself as a combination of a wise friend and an experienced grandfather/grandmoth